# Selenium 이용한 동적 크롤링

In [7]:
from selenium.webdriver.common.by import By
from selenium import webdriver
import sys
import time
import warnings
warnings.filterwarnings('ignore')

# # 옵션 생성
options = webdriver.ChromeOptions()
# # 창 숨기는 옵션 추가
options.add_argument("headless")
#지정한 user-agent로 설정합니다.
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"
options.add_argument('user-agent=' + user_agent)

# 드라이버 열기
driver = webdriver.Chrome('chromedriver.exe', options=options)
# driver = webdriver.Chrome('chromedriver.exe')


# 구글에 삼성전자 검색 -> 뉴스 -> 지난 1개월
url = 'https://www.google.com/search?q=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&tbm=nws&sxsrf=AJOqlzUEJoq0SuwDJ75imZRaB1y76Pq-rg:1673621191460&source=lnt&tbs=qdr:m&sa=X&ved=2ahUKEwim2-vk5MT8AhVYMXAKHfPbABAQpwV6BAgBEBw&biw=1280&bih=609&dpr=1.5'
driver.get(url)

# 검색창 지우기 
driver.find_element(By.CLASS_NAME, 'gLFyf').clear()

# 검색창에 검색어 입력하기
search_item = "카카오"
driver.find_element(By.CLASS_NAME, 'gLFyf').send_keys(search_item)

# 클릭하기
driver.find_element(By.CLASS_NAME, 'Tg7LZd').click()

# 뉴스 제목 가져오기
# for i in range(1, 11):
#     xpath = f'//*[@id="rso"]/div/div/div[{i}]/div/div/a/div/div[2]/div[2]'
#     text = driver.find_element(By.XPATH, xpath).text
#     print(text)
    
# 다음 페이지 클릭 
page = 4
xpath = f'//*[@id="botstuff"]/div/div[2]/table/tbody/tr/td[{page+1}]/a'
driver.find_element(By.XPATH, xpath).click()

# 총 페이지 개수랑 , 한 페이지당 기사 개수를 알 수가 없어서 while + try-except 문으로 처리했다. 
#  1페이지부터 끝 페이지까지 제목 쭉 불러오기
# 12페이지 넘어가면 13으로 고정됨. 
titles = []
page = 1
is_final = True

while True:
    try:
        print(f"page{page} try", end=' ')
        try:
            if page <= 12:
                xpath = f'//*[@id="botstuff"]/div/div[2]/table/tbody/tr/td[{page+1}]/a'
                driver.find_element(By.XPATH, xpath).click()
            else: 
                # 12페이지 넘어가면 xpath 고정됨
                xpath = f'//*[@id="botstuff"]/div/div[2]/table/tbody/tr/td[{13}]/a'
                driver.find_element(By.XPATH, xpath).click()
        except:
            sys.exit() # 마지막 페이지 넘어가면 완전히 종료 
            
        table_count = 1
        while True:
            try:
                xpath = f'//*[@id="rso"]/div/div/div[{table_count}]/div/div/a/div/div[2]/div[2]'
                text = driver.find_element(By.XPATH, xpath).text
                titles.append(text)
                table_count += 1
            except:
                break
        if table_count > 1:
            print(f"page{page} ok , table count : {table_count}")
        else:
            print(f"page{page} not ok")
        page += 1
        
        time.sleep(0.5)
        
    except:
        break
        
print(f"avg table per page : {len(titles) / (page - 1)}")

# 창 종료.
driver.quit()

C:\Users\Hi\AppData\Local\Temp\ipykernel_1404\1057774138.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe', options=options)


page1 try page1 ok , table count : 11
page2 try page2 ok , table count : 11
page3 try page3 ok , table count : 10
page4 try page4 ok , table count : 11
page5 try page5 ok , table count : 11
page6 try page6 ok , table count : 11
page7 try page7 ok , table count : 10
page8 try page8 ok , table count : 11
page9 try page9 ok , table count : 11
page10 try page10 ok , table count : 11
page11 try page11 ok , table count : 11
page12 try page12 ok , table count : 11
page13 try page13 ok , table count : 11
page14 try page14 ok , table count : 11
page15 try page15 ok , table count : 11
page16 try page16 ok , table count : 7
page17 try avg table per page : 9.625


In [21]:
# 함수화 , 최근 한 달간의 제목을 가져오는 함수

from selenium.webdriver.common.by import By
from selenium import webdriver
import sys
import time
import warnings
warnings.filterwarnings('ignore')

def get_titles_recent30(stock_name, verbose1=True, verbose2=True): 
    """
    stock_name : 검색할 종목 이름 -> Str,
    verbose1 : 개별 페이지 결과를 출력할지 여부 -> Bool,
    verboss2 : 결과를 출력할지 여부 ->  Bool,
    return : 최근 한 달간 제목 리스트 -> List.
    """
    # # 옵션 생성
    options = webdriver.ChromeOptions()
    # # 창 숨기는 옵션 추가
    options.add_argument("headless")
    #지정한 user-agent로 설정합니다.
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"
    options.add_argument('user-agent=' + user_agent)

    # 드라이버 열기
    driver = webdriver.Chrome('chromedriver.exe', options=options)
    # driver = webdriver.Chrome('chromedriver.exe')


    # 구글에 삼성전자 검색 -> 뉴스 -> 지난 1개월
    url = 'https://www.google.com/search?q=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&tbm=nws&sxsrf=AJOqlzUEJoq0SuwDJ75imZRaB1y76Pq-rg:1673621191460&source=lnt&tbs=qdr:m&sa=X&ved=2ahUKEwim2-vk5MT8AhVYMXAKHfPbABAQpwV6BAgBEBw&biw=1280&bih=609&dpr=1.5'
    driver.get(url)

    # 검색창 지우기 
    driver.find_element(By.CLASS_NAME, 'gLFyf').clear()

    # 검색창에 검색어 입력하기
    search_item = stock_name
    driver.find_element(By.CLASS_NAME, 'gLFyf').send_keys(search_item)

    # 클릭하기
    driver.find_element(By.CLASS_NAME, 'Tg7LZd').click()

    # 뉴스 제목 가져오기
    # for i in range(1, 11):
    #     xpath = f'//*[@id="rso"]/div/div/div[{i}]/div/div/a/div/div[2]/div[2]'
    #     text = driver.find_element(By.XPATH, xpath).text
    #     print(text)

    # 다음 페이지 클릭 
    page = 4
    xpath = f'//*[@id="botstuff"]/div/div[2]/table/tbody/tr/td[{page+1}]/a'
    driver.find_element(By.XPATH, xpath).click()

    # 총 페이지 개수랑 , 한 페이지당 기사 개수를 알 수가 없어서 while + try-except 문으로 처리했다. 
    #  1페이지부터 끝 페이지까지 제목 쭉 불러오기
    # 12페이지 넘어가면 13으로 고정됨. 
    titles = []
    page = 1
    is_final = True

    while True:
        try:
            if verbose1:
                print(f"page{page} try", end=' ')
                
            try:
                if page <= 12:
                    xpath = f'//*[@id="botstuff"]/div/div[2]/table/tbody/tr/td[{page+1}]/a'
                    driver.find_element(By.XPATH, xpath).click()
                else: 
                    # 12페이지 넘어가면 xpath 고정됨
                    xpath = f'//*[@id="botstuff"]/div/div[2]/table/tbody/tr/td[{13}]/a'
                    driver.find_element(By.XPATH, xpath).click()
            except:
                sys.exit() # 마지막 페이지 넘어가면 완전히 종료 

            table_count = 1
            while True:
                try:
                    xpath = f'//*[@id="rso"]/div/div/div[{table_count}]/div/div/a/div/div[2]/div[2]'
                    text = driver.find_element(By.XPATH, xpath).text
                    titles.append(text)
                    table_count += 1
                except:
                    break
            if verbose1:
                if table_count > 1:
                    print(f"page{page} ok , table count : {table_count}")
                else:
                    print(f"page{page} not ok")
            page += 1

            time.sleep(0.5)

        except:
            break
            
    if verbose2:
        print(f"avg table per page : {len(titles) / (page - 1)}")

    # 창 종료.
    driver.quit()
    
    return titles

In [22]:
?get_titles_recent30

In [24]:
title_kakao = get_titles_recent30('카카오', verbose1=False, verbose2=True)
# 종목당 30초정도 걸리네. 

avg table per page : 9.75


# rss_feedparser을 이용한 뉴스 크롤링

In [30]:
import feedparser

def get_titles_by_ssl(start, end, search_name, verbose1=True, verbose2=True):
    """
    start : 데이터를 조회할 시작 날짜 -> Str (ex:2022-06-01),
    end : 데이터를 조회할 마지막 날짜 -> Str (ex:2022-10-31),
    search_name  : 검색할 이름 -> Str, 
    verbose1 : 과정을 출력할지 여부 -> Bool,
    verbose2 : 결과를 출력할지 여부 -> Bool,
    return : 뉴스 제목 리스트 -> List
    """

    ssl_url = f'https://news.google.com/news?hl=ko&gl=kr&ie=UTF-8&q={search_name}+after:{start}+before:{end}&output=rss'

    parse_res = feedparser.parse(ssl_url)

    titles = []
    
    for p in parse_res.entries:
        if verbose1:
            print(p.title)
        titles.append(p.title)
        
    if verbose2:
        print(len(titles))
    return titles

In [31]:
?get_titles_by_ssl

In [32]:
kakao_titles = get_titles_by_ssl(start='2022-01-01', end='2022-10-31', search_name='카카오', verbose1=False, verbose2=True)

100


In [34]:
kakao_titles

['카카오톡·버스·지하철·페이지·티…단번에 끊겼다 - 한겨레',
 "'카카오 화재' 보상 방안은 어떻게 되나? - BBC News 코리아",
 '‘복구중’ 카카오, 톡으로 상황 공유…앱 다운 1위는 ‘라인’ - 서울신문',
 "카카오엔터, 작가 정산 확인 사이트 '파트너 포털' 정식 운영 - 뉴스저널리즘",
 '[영상] 카카오 “무료 이용자도 보상…SK C&C와 잘잘못 안 따질 것” - 한겨레',
 '카카오뱅크 주가 추락에 우리사주 “신불자 위기” 호소 - 뉴스저널리즘',
 '[속보] 카카오톡 오류…다음·카카오T 등 카카오서비스 동시 마비 - 서울신문',
 '카톡 장애 원인은 \'내부 시스템 오류\'…카카오 측 "현재 모두 정상화" - 위키리크스한국',
 "카카오 대란…손실보전 피해 간 독점 플랫폼에 '사회적 책임' 묻는다 - 한겨레",
 '창업 시절 배고픔 잊고, 거만해진 카카오 - 아틀라스뉴스',
 "2년 전 '카카오 먹통 방지법안', 누가 왜 막았나 - 한겨레",
 "네이버·카카오 주가 '반토막'…올해 시가총액 63조 증발 - 한겨레",
 '카카오 무료 이용자 보상은 어쩌나 [선한결의 IT포커스] - 한국경제',
 "카카오, 2주 만에 구글에 '백기'…카톡 '결제 아웃링크' 뺀다 - 한겨레",
 '카카오, 점유율 98% 카톡 앞세워 사업 확장… “독점 폐해 낳아” - 동아일보',
 '카카오 먹통 사태, 서비스 장애 127시간 30분으로 공식화 - 경향신문',
 '카카오톡이 멈추자 벌어진 일 - 뉴닉',
 '카카오모빌리티 “지분 안 팔겠다”…탄력요금제 희망 봤나 - 한겨레',
 '카카오그룹株 \'대폭락의 날\'…-80% 개미들 "살려달라" 아우성 - 머니투데이',
 '카카오 먹통 사태 ‘플랫폼 독점’ 비판에도 ‘시장에 맡기라’는 경제지 - 미디어오늘',
 "“카카오에 정 떨어졌어요”…카카오서 탈출, 라인·텔레그램 가입 '러시' - 데일리포스트-스타트업워치",
 '“카카오 독점” 문제라면서 여전히 자율규제?…안일한 공정위 - 한겨레',
 "성장둔화 우려 카카